# Documentation assistent

This notebook demonstrates a documentation assistent:
1. Video-to-protocol conversion using Vertex AI
2. Without feeding the Gen AI model with knowledge


In [1]:
# %pip install google-cloud-storage
# %pip install --upgrade --user --quiet google-cloud-aiplatform

In [2]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import configparser
import os

from IPython.display import Markdown

# path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
# sys.path.append(str(path_to_append))
# import documentation_agent as doc_agent

config = configparser.ConfigParser()
config.read("../secrets.ini")

['../secrets.ini']

In [3]:
import configparser

import vertexai

config = configparser.ConfigParser()
config.read("../secrets.ini")

PROJECT_ID = config["DEFAULT"]["PROJECT_ID"]
vertexai.init(project=PROJECT_ID, location="europe-west9")  # europe-west9 is Paris

In [4]:
# Upload video with Vertex AI

from google.cloud import storage

# 1. Set Project ID
os.environ["GOOGLE_CLOUD_PROJECT"] = config["DEFAULT"]["PROJECT_ID"]

# 2. Initialize Cloud Storage client
storage_client = storage.Client()
bucket_name = "mannlab_videos"
bucket = storage_client.bucket(bucket_name)

# 3. Upload the video to Cloud Storage
video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/Disconnect_IonOpticks_column_from_sample_line.mp4"  # Path to your local video file
blob_name = (
    "Connect_ionOpticks_column.mp4"  # Desired name for the video in Cloud Storage
)
blob = bucket.blob(blob_name)
blob.upload_from_filename(video_path)

# 4. Get the Cloud Storage URI
video_uri = f"gs://{bucket_name}/{blob_name}"
print(f"Video uploaded to: {video_uri}")

Video uploaded to: gs://mannlab_videos/Connect_ionOpticks_column.mp4


In [5]:
prompt = """
You are a highly observant research assistant in Professor Matthias Mann's proteomics and mass spectrometry laboratory. Your expertise lies in detailed documentation of experimental procedures.

Analyze the video and reconstruct a step-by-step protocol by focusing on the actions in the video. Focus on user interactions with equipment, devices, and software. The goal is a clear, concise, unambiguous protocol reproducible by someone with no prior knowledge. "Think aloud" as if you were the researcher in the video that describes their work. Describe what you see at every secound. Take deep breath and think step-by-step. Answer direct.

For each action, describe:

* **Timestamp:** [timestamp]
* **Action:** [Specific Action/Change Observed (Include details of tools used, and observable results. (e.g., opening a lid, pressing a button, turning a knob, screwing/unscrewing, connecting/disconnecting, etc.))]

**Example:**
[02:15] timsControl Software: Mode changed from "Operate" to "Standby" by clicking the power button.
[03:45] Ion Source: NanoViper connector disconnected by unscrewing the nut counterclockwise.

"""

In [6]:
# Code from: https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/video-understanding
import vertexai
from vertexai.generative_models import GenerativeModel, Part

inputs = [
    prompt,
    "Input Video:",
    Part.from_uri(video_uri, mime_type="video/mp4"),
    "Observations:",
]
model = GenerativeModel("gemini-1.5-flash-002")
response = model.generate_content(inputs, generation_config={"temperature": 0})
observation = response.text
print(response.usage_metadata)
Markdown(observation)

prompt_token_count: 14221
candidates_token_count: 470
total_token_count: 14691
prompt_tokens_details {
  modality: VIDEO
  token_count: 13965
}
prompt_tokens_details {
  modality: TEXT
  token_count: 256
}
candidates_tokens_details {
  modality: TEXT
  token_count: 470
}



Okay, let's break down the video step-by-step.  I'll describe each action as it happens.

**[00:00-00:07]**: The video starts with a view of a computer screen displaying a chromatogram and other data, likely from mass spectrometry software. A blue nitrile glove lies on the desk next to a keyboard and mouse. The computer is connected to a black desktop tower labeled "Innacopia".  No actions are taken during this time.

**[00:08-00:11]**: The camera pans to show the laboratory environment, including other equipment and lab benches.  The camera moves to a different bench.

**[00:12-00:13]**: The camera focuses on a piece of equipment, possibly a nano-flow HPLC system.  The researcher's hand approaches the equipment.

**[00:14-00:32]**: The researcher connects a very thin capillary tubing (likely a nano-ESI emitter) to the ion source of a mass spectrometer. This involves careful manipulation to ensure a secure connection. The emitter is guided into a small opening and appears to be secured by a small screw mechanism.

**[00:33-00:39]**: The researcher uses a pair of needle-nose pliers to further secure the capillary tubing connection at the ion source.  This suggests a need for precise positioning and a firm connection.

**[00:40-00:43]**: The researcher removes the excess length of the capillary tubing and places it into a small yellow waste container.

**[00:44-00:49]**: The camera pans away from the mass spectrometer, showing a general view of the lab again, and then back to the computer. No further actions are performed on the equipment.


This protocol describes the connection of a nano-ESI emitter to a mass spectrometer's ion source.  The use of pliers suggests that a very secure connection is crucial for successful analysis. The careful handling of the emitter indicates its fragility and the importance of preventing damage.  Further steps would involve sample loading and initiating the mass spectrometry analysis via the computer software.
